In [20]:
import sys
sys.path.append('../../')
from data.raw_datas import gemini_default_prompt
import google.generativeai as genai
from dotenv import load_dotenv
from pathlib import Path
import json
import os

load_dotenv()

# 제미나이 불러오기
api_key = os.getenv("GEMNI_API_KEY")
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.0-flash')


# telegram 필터링된 데이터 불러오기
filtered_telegram_data_path = "../data/filtered_telegram_data.json"
with open(filtered_telegram_data_path, 'r', encoding='utf-8') as f:
    filtered_telegram_data = json.load(f)


In [39]:
def ask_gemini(data):
    prompt = gemini_default_prompt + data
    response = model.generate_content(prompt)
    response_filter = response.text[7:-3].strip()
    try:
        return json.loads(response_filter)
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 오류: {e}")
        return None



In [40]:
datas = {}
should_break = False  # 루프 종료 플래그

for sector in filtered_telegram_data.keys():
    if should_break:
        break  # 외부 루프 종료
        
    if sector not in datas:
        datas[sector] = {}
        
    for date in filtered_telegram_data[sector]:
        try:
            print(sector, date)
            datas[sector][date] = ask_gemini(str(dict(filtered_telegram_data[sector][date])))
            should_break = True  # 플래그 설정
            break  # 내부 루프 종료
        except Exception as e:
            print(f"오류 발생: {type(e).__name__} - {str(e)}")
        
print(datas)

전선 2025-05-27
{'전선': {'2025-05-27': '{\n    "scores": {\n        "중앙기획전략부": 70,\n        "COINNESS News Feed (코인니스)": 50,\n        "에테르의 일본&미국 리서치": 100,\n        "💯똥밭에 굴러도 주식판": 60,\n        "TNBfolio": 70,\n        "키움증권 미국주식 톡톡": 50,\n        "Yeouido Lab_여의도 톺아보기": 70,\n        "참새선생의 주식공부방": 40,\n        "탑클래스 투자": 70,\n        "Palo Alto 94301": 80,\n        "Brain and Body Research": 70,\n        "Pluto Research": 60,\n        "대신 신성장산업 & 유틸리티": 80,\n        "퀀텀 알고리즘": 70,\n        "뺀지뤼의 SKLab.(스크랩)": 50,\n        "하나증권 리서치": 30,\n        "여의도스토리 Ver2.0": 50,\n        "Hodolry의 텔레그램": 70,\n        "그로쓰리서치(Growth Research) [독립리서치]": 80,\n        "키움증권 리서치센터": 90,\n        "와우넷 한중연": 40,\n        "현대차증권_리서치센터_채널": 80,\n        "이지스 리서치 & 모멘텀투자클럽": 70,\n        "Nihil\'s view of data & information": 50,\n        "유안타증권 리서치센터": 60\n    },\n    "summary": "전반적으로 시장은 에너지, 방산, 조선, 전력기기 분야에 대한 긍정적인 전망이 우세하며, 특히 AI와 관련된 전력 수요 증가와 데이터센터 투자 확대가 주요 화두로 떠오르고 있다. 원자력 발전 재개 및 SMR(소형모듈원자로) 사업 추

In [25]:
gemini_result_data_path = "../data/gemini_result_data.json"
